In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# oversampling packages
from imblearn.combine import SMOTEENN
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import RandomOverSampler

#optimisation
from bayes_opt import BayesianOptimization

# model building
import keras
import tensorflow
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Embedding, SimpleRNN , concatenate, Lambda, Conv1D, MaxPooling1D, Flatten
from keras.layers import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn import linear_model, model_selection

In [3]:
X_train = pd.read_csv('../datasets/word2vec/train_data_imputed_FINAL.csv')
X_test = pd.read_csv('../datasets/word2vec/test_data_imputed_FINAL.csv')
y_train = pd.read_csv('../datasets/word2vec/y_train_FINAL.csv')
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv')

In [ ]:
def rnn_tuner( batch_size, neurons, activation, optimizer, dropout, dropout_rate, learning_rate, epochs, layers1, layers2):

    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                  'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                    'elu', 'exponential', 'LeakyReLU','relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def create_model():
        tuned_model = Sequential()

        for i in range(layers1): #varying first set of hidden layers
            tuned_model.add(Dense(neurons, activation=activation))

        if dropout > 0.5:
            tuned_model.add(Dropout(dropout_rate, seed=42))

        for i in range(layers2): #varying second set of hidden layers
            tuned_model.add(Dense(neurons, activation=activation))

        tuned_model.add(Dense(1, activation='sigmoid')) #output layer

        tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
       # tuned_model = tuned_model.fit(X_train_smenn, y_train_smenn, epochs=20)
       # tuned_model.summary()

        return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_RNN_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, 
                            fit_params={'callbacks':[es]}, error_score='raise',
                           scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
ros = RandomOverSampler(sampling_strategy = 1, random_state=42)

In [ ]:
X_train_over, y_train_over = ros.fit_resample(X_train, y_train)

In [ ]:
params_rnn ={
    'neurons': (32, 256),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.001, 0.01),
    'batch_size':(100, 1000),
    'epochs':(10, 30),
    'layers1':(1,3),
    'layers2':(1,4),
    'dropout':(0,1),
    'dropout_rate':(0,0.4)
}

In [ ]:
rnn_bo = BayesianOptimization(rnn_tuner, params_rnn, random_state=42)
rnn_bo.maximize(init_points=40, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_4128\1000544679.py:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)


142/142 [==============================] - 0s 1ms/step
| 1         | 0.8179    | 3.371     | 955.6     | 0.732     | 0.2395    | 13.12     | 1.312     | 1.174     | 0.008796  | 166.6     | 4.957     |


C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_4128\1000544679.py:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)


142/142 [==============================] - 1s 3ms/step
| 2         | 0.7633    | 0.1853    | 972.9     | 0.8324    | 0.08494   | 13.64     | 1.367     | 1.913     | 0.005723  | 128.8     | 2.039     |


...

142/142 [==============================] - 2s 7ms/step
| 42        | 0.795     | 7.997     | 740.5     | 0.2353    | 0.1587    | 12.02     | 1.746     | 1.015     | 0.005883  | 43.6      | 0.7721    |


C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_4128\1000544679.py:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)


142/142 [==============================] - 2s 6ms/step
| 43        | 0.5339    | 8.098     | 290.1     | 0.74      | 0.3059    | 17.67     | 2.299     | 1.363     | 0.004477  | 113.9     | 3.878     |


C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_4128\1000544679.py:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)


142/142 [==============================] - 2s 9ms/step
| 44        | 0.1416    | 4.03      | 842.3     | 0.8029    | 0.06778   | 10.7      | 1.754     | 2.033     | 0.003354  | 215.7     | 6.856     |


In [ ]:
{'activation': 4, 'batch_size': 170.6, 'dropout': .102535, 'dropout_rate': 0.3851, 'epochs' : 27, 'layers1': 2, 'layers2':2 , 
 'learning_rate': 0.00256, 
'neurons': 67, 'optimizer': 1.752, }

{'activation': 6,
 'batch_size': 175.7,
 'dropout': 0.1616,
 'dropout_rate': 0.2696,
 'epochs': 22.13,
 'layers1': 1.028,
 'layers2': 1.304,
 'learning_rate': 0.006972,
 'neurons': 33.13,
 'optimizer': 1.126}

In [ ]:
new_model = Sequential()
new_model.add(Dense(241.8, activation='relu'))
new_model.add(Dropout(0.06795))
new_model.add(Dense(241.8, activation='relu'))
new_model.add(Dropout(0.06795))
new_model.add(Dense(241.8, activation='relu'))
new_model.add(Dropout(0.06795))
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
new_model.compile(loss='binary_crossentropy', optimizer = Adam(learning_rate=0.00256), metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])

In [ ]:
ros = RandomOverSampler(sampling_strategy = 1, random_state=42)
X_res, y_res = ros.fit_resample(X_train, y_train)

In [ ]:
new_model.fit(X_res, y_res, epochs=27, batch_size=171)

Epoch 1/27
133/133 [==============================] - 4s 11ms/step - loss: 0.6713 - accuracy: 0.7617 - precision_667: 0.7508 - recall_667: 0.7834 - auc_667: 0.8365
Epoch 2/27
133/133 [==============================] - 2s 11ms/step - loss: 0.1556 - accuracy: 0.9427 - precision_667: 0.9257 - recall_667: 0.9626 - auc_667: 0.9814
Epoch 3/27
133/133 [==============================] - 2s 11ms/step - loss: 0.0902 - accuracy: 0.9691 - precision_667: 0.9552 - recall_667: 0.9842 - auc_667: 0.9921
Epoch 4/27
133/133 [==============================] - 1s 10ms/step - loss: 0.0754 - accuracy: 0.9746 - precision_667: 0.9629 - recall_667: 0.9873 - auc_667: 0.9942
Epoch 5/27
133/133 [==============================] - 1s 10ms/step - loss: 0.0439 - accuracy: 0.9859 - precision_667: 0.9787 - recall_667: 0.9934 - auc_667: 0.9976
Epoch 6/27
133/133 [==============================] - 1s 10ms/step - loss: 0.0315 - accuracy: 0.9896 - precision_667: 0.9848 - recall_667: 0.9944 - auc_667: 0.9985
Epoch 7/27
133/1

In [ ]:
train_score = new_model.evaluate(X_res, y_res)
print(train_score)

706/706 [==============================] - 3s 4ms/step - loss: 0.0137 - accuracy: 0.9961 - precision_667: 0.9923 - recall_667: 1.0000 - auc_667: 0.9994
[0.013684666715562344, 0.9961037635803223, 0.9922677874565125, 1.0, 0.9994206428527832]


In [ ]:
pred = new_model.predict(X_test)

90/90 [==============================] - 0s 3ms/step


In [ ]:
y_pred = np.where(pred < 0.5, 0, 1)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[2769,   55],
       [   9,   47]], dtype=int64)

In [ ]:
f1_score(y_test, y_pred)

0.5949367088607594

## CNN Bayes Optimisation

In [ ]:
def rnn_tuner( batch_size, neurons, activation, optimizer, dropout, dropout_rate, dropout2, dropout_rate1, learning_rate, epochs, layers1, layers2, filters, kernel, pool_size, strides):

    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                  'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                    'elu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    filters = round(filters)
    kernel = round(kernel)
    pool_size = round(pool_size)
    if round(strides) == 0:
      strides = None
    else:
      strides = round(strides)
    
    def create_model():
        tuned_model = Sequential()
        tuned_model.add(Conv1D(filters, kernel, activation=activation, input_shape=(X_train_over.shape[1], 1)))
        tuned_model.add(MaxPooling1D(pool_size=pool_size, strides=strides))
        tuned_model.add(Flatten())

        for i in range(layers1): #varying first set of hidden layers
            tuned_model.add(Dense(neurons, activation=activation))

            if dropout > 0.5:
                tuned_model.add(Dropout(dropout_rate, seed=42))

        for i in range(layers2): #varying second set of hidden layers
            tuned_model.add(Dense(neurons, activation=activation))

            if dropout2 > 0.5:
              tuned_model.add(Dropout(dropout_rate1, seed=42))

        tuned_model.add(Dense(1, activation='sigmoid')) #output layer

        tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
       # tuned_model = tuned_model.fit(X_train_smenn, y_train_smenn, epochs=20)
       # tuned_model.summary()

        return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_RNN_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, 
                            fit_params={'callbacks':[es]}, error_score='raise',
                           scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
ros = RandomOverSampler(sampling_strategy = 1, random_state=42)

In [ ]:
X_train_over, y_train_over = ros.fit_resample(X_train, y_train)

In [ ]:
params_rnn ={
    'neurons': (32, 256),
    'activation':(0, 5),
    'optimizer':(0,7),
    'learning_rate':(0.001, 0.01),
    'batch_size':(100, 1000),
    'epochs':(10, 30),
    'layers1':(1,3),
    'layers2':(1,4),
    'dropout':(0,1),
    'dropout_rate':(0,0.4),
    'dropout2': (0, 1),
    'dropout_rate1' : (0, 0.4),
    'filters': (16, 64),
    'kernel': (1, 4),
    'pool_size': (1, 4),
    'strides': (0, 4)
}

In [ ]:
rnn_bo = BayesianOptimization(rnn_tuner, params_rnn, random_state=42)
rnn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropout2  | dropou... | dropou... |  epochs   |  filters  |  kernel   |  layers1  |  layers2  | learni... |  neurons  | optimizer | pool_size |  strides  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 2s 16ms/step
| 1         | 0.1416    | 1.873     | 955.6     | 0.732     | 0.5987    | 0.06241   | 0.0624    | 11.16     | 57.58     | 2.803     | 2.416     | 1.062     | 0.009729  | 218.5     | 1.486     | 1.545     | 0.7336    |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 2s 11ms/step
| 2         | 0.1526    | 1.521     | 572.3     | 0.4319    | 0.2912    | 0.2447    | 0.0558    | 15.84     | 33.59     | 2.368     | 2.57      | 1.599     | 0.005628  | 164.7     | 0.3252    | 2.823     | 0.6821    |


...

142/142 [==============================] - 1s 7ms/step
| 27        | 0.9356    | 3.205     | 263.7     | 0.6181    | 0.4978    | 0.3217    | 0.2936    | 27.93     | 54.15     | 2.186     | 2.611     | 2.576     | 0.003667  | 113.8     | 1.923     | 2.646     | 3.171     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 2s 11ms/step
| 28        | 0.1467    | 1.906     | 260.0     | 0.0856    | 0.7697    | 0.2045    | 0.3538    | 16.74     | 57.28     | 1.875     | 2.276     | 1.794     | 0.006772  | 104.3     | 1.267     | 3.558     | 1.645     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 6ms/step
| 29        | 0.917     | 3.51      | 268.9     | 1.0       | 0.556     | 0.4       | 0.3176    | 28.88     | 57.39     | 1.733     | 1.946     | 1.931     | 0.002523  | 118.6     | 2.08      | 2.852     | 2.681     |


In [ ]:
def rnn_tuner( batch_size, neurons, activation, optimizer, learning_rate, epochs, filters, kernel, pool_size, strides):

    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                  'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                    'elu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    filters = round(filters)
    kernel = round(kernel)
    pool_size = round(pool_size)
    if round(strides) == 0:
      strides = None
    else:
      strides = round(strides)
    
    def create_model():
        tuned_model = Sequential()
        tuned_model.add(Conv1D(filters, kernel, activation=activation, input_shape=(X_train_over.shape[1], 1)))
        tuned_model.add(MaxPooling1D(pool_size=pool_size, strides=strides))
        tuned_model.add(Flatten())

        tuned_model.add(Dense(256, activation='relu'))
        tuned_model.add(Dropout(0.2))
        tuned_model.add(Dense(128, activation='relu'))
        tuned_model.add(Dropout(0.2))
        tuned_model.add(Dense(64, activation='relu'))
        tuned_model.add(Dropout(0.2))
        tuned_model.add(Dense(32, activation='relu'))
        tuned_model.add(Dropout(0.2))

        tuned_model.add(Dense(1, activation='sigmoid')) #output layer

        tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
       # tuned_model = tuned_model.fit(X_train_smenn, y_train_smenn, epochs=20)
       # tuned_model.summary()

        return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_RNN_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, 
                            fit_params={'callbacks':[es]}, error_score='raise',
                           scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
ros = RandomOverSampler(sampling_strategy = 1, random_state=42)
X_train_over, y_train_over = ros.fit_resample(X_train, y_train)

In [ ]:
params_rnn ={
    'neurons': (32, 256),
    'activation':(0, 6),
    'optimizer':(0, 5),
    'learning_rate':(0.001, 0.01),
    'batch_size':(100, 1000),
    'epochs':(10, 30),
    'filters': (16, 64),
    'kernel': (1, 4),
    'pool_size': (1, 4),
    'strides': (0, 4)
}

In [ ]:
rnn_bo = BayesianOptimization(rnn_tuner, params_rnn, random_state=42)
rnn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   |  filters  |  kernel   | learni... |  neurons  | optimizer | pool_size |  strides  |
-------------------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 9ms/step
| 1         | 0.3078    | 2.247     | 955.6     | 24.64     | 44.74     | 1.468     | 0.002404  | 45.01     | 4.331     | 2.803     | 2.832     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 7ms/step
| 2         | 0.1426    | 0.1235    | 972.9     | 26.65     | 26.19     | 1.545     | 0.002651  | 100.2     | 2.624     | 2.296     | 1.165     |


...

142/142 [==============================] - 1s 6ms/step
| 27        | 0.01537   | 0.6007    | 610.2     | 10.79     | 30.52     | 3.505     | 0.008133  | 249.6     | 0.1456    | 2.655     | 0.4173    |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 9ms/step
| 28        | 0.8812    | 5.398     | 290.1     | 24.8      | 52.7      | 2.15      | 0.006845  | 59.11     | 1.932     | 2.097     | 2.216     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 4ms/step
| 29        | 0.9098    | 2.687     | 842.3     | 26.06     | 24.13     | 1.105     | 0.004395  | 109.2     | 1.308     | 3.46      | 3.918     |
